<a href="https://colab.research.google.com/github/Dembelinho/Langchain_Agents_RAG_LLM/blob/main/Cohere_Semantic_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Fireworks

In [ ]:
!pip install -qU langchain-fireworks

# Cohere

---------------Semantic Search using cohere-----------------

- Install Cohere for embeddings, Umap to reduce embeddings to 2 dimensions, Altair for visualization, Annoy for approximate nearest neighbor search

In [ ]:
!pip install "cohere<5" umap-learn altair annoy datasets tqdm

In [ ]:
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [ ]:
from google.colab import userdata
api_key =userdata.get('COHERE_API_KEY')

set up the cohere client

In [ ]:
model_name = "embed-multilingual-v3.0"
input_type_embed = "search_document"
co = cohere.Client(api_key)

In [ ]:
!pip install langchain_community

In [ ]:
!pip install pymupdf

In [ ]:
!pip install -U langchain-text-splitters

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
data = PyMuPDFLoader("/content/thingsboars.pdf").load_and_split()

In [ ]:
len(data)

3

In [ ]:
for document in data:
    page_content = document.page_content.replace('\n', ' ').split('.')
    print(page_content)


["Home page  La page d'accueil de ThingsBoard est le tableau de bord principal de la plateforme IoT ThingsBoard", '  Elle offre un emplacement central où vous pouvez consulter vos appareils, vos données et vos  applications', " La page d'accueil est divisée en plusieurs sections, notamment  Tableaux de bord : Cette section affiche une liste de vos tableaux de bord", " Les tableaux de bord sont  utilisés pour visualiser vos données d'une manière facile à comprendre", '  Appareils : Cette section affiche une liste de vos appareils', ' Les appareils sont des entités physiques  ou virtuelles qui collectent des données dans le monde réel', '  Flux de données : Cette section affiche une liste de vos flux de données', ' Les flux de données sont les  canaux par lesquels les données sont envoyées des appareils au ThingsBoard', '  Applications : Cette section affiche une liste de vos applications', ' Les applications sont des logiciels  que vous pouvez utiliser pour interagir avec vos données', 

In [ ]:
# Create a pandas dataframe
df = pd.DataFrame(page_content, columns=['text'])

# Preview the data to ensure it has loaded correctly
print(df)

                                                                                                                                                                                                                                              text
0                                                                                           Pour ajouter un appareil, vous pouvez cliquer sur l'icône "+" dans le coin supérieur droit du tableau,  puis sélectionner "Ajouter un nouvel appareil"
1                                                                                                                                                              Vous devrez fournir un nom de dispositif et,  éventuellement, un type de dispositif
2                                                                                         Une fois que vous avez ajouté un appareil, vous pouvez gérer ses informations d'identification en  cliquant sur l'onglet "Informations d'identification"
3                           

In [ ]:
# Get the embeddings
embeds = co.embed(texts=list(df['text']),
                  model=model_name,
                  input_type=input_type_embed).embeddings

In [ ]:
# Check the dimensions of the embeddings
embeds = np.array(embeds)
embeds.shape

(24, 1024)

# Search using an index and nearest neighbor search

Let's now use Annoy to build an index that stores the embeddings in a way that is optimized for fast search. This approach scales well to a large number of texts (other options include Faiss, ScaNN, and PyNNDescent).

After building the index, we can use it to retrieve the nearest neighbors either of existing questions , or of new questions that we embed .

In [ ]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

# Find the neighbors of an example from the dataset

If we're only interested in measuring the distance between the questions in the dataset (no outside queries), a simple way is to calculate the distance between every pair of embeddings we have.

In [ ]:
# Choose an example (we'll retrieve others similar to it)
example_id = 6

# Retrieve nearest neighbors
similar_item_ids = search_index.get_nns_by_item(example_id,10,
                                                include_distances=True)
# Format and print the text and distances
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]}).drop(example_id)

print(f"Question:'{df.iloc[example_id]['text']}'\nNearest neighbors:")
results

Question:'  La page ThingsBoard Devices est un outil puissant pour gérer vos appareils'
Nearest neighbors:


,texts,distance
10,Les chaînes de règles constituent un outil puissant pour le traitement des données provenant des appareils IoT,0.771786
8,Une chaîne de règles ThingsBoard est une séquence de nœuds de règles qui sont exécutés dans l'ordre,0.778087
7,"Vous pouvez l'utiliser pour assurer le suivi de vos appareils, consulter leurs données et gérer leurs informations d'identification",0.811441
2,"Une fois que vous avez ajouté un appareil, vous pouvez gérer ses informations d'identification en cliquant sur l'onglet ""Informations d'identification""",0.837800
0,"Pour ajouter un appareil, vous pouvez cliquer sur l'icône ""+"" dans le coin supérieur droit du tableau, puis sélectionner ""Ajouter un nouvel appareil""",0.840088
5,Ces onglets affichent une liste de toutes les données collectées à partir de l'appareil,0.840252
4,"Pour parcourir les attributs, la télémétrie, les alarmes, les événements et les relations de l'appareil, vous pouvez cliquer sur les onglets correspondants",0.842978
22,Cela peut être utile pour automatiser des tâches ou pour traiter régulièrement des données provenant d'appareils IoT,0.851465
17,Voici d'autres choses que vous pouvez faire avec les chaînes de règles de ThingsBoard : * Exporter et importer des chaînes de règles : Vous pouvez exporter et importer des chaînes de règles depuis et vers la plateforme ThingsBoard,0.902468


# Find the neighbors of a user query

We're not limited to searching using existing items. If we get a query, we can embed it and find its nearest neighbors from the dataset.

In [ ]:
query = "Comment les chaînes de règles peuvent être utilisées pour déclencher des actions?"
input_type_query = "search_query"

# Get the query's embedding
query_embed = co.embed(texts=[query],
                  model=model_name,
                  input_type=input_type_query).embeddings

# Retrieve the nearest neighbors
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                include_distances=True)
# Format the results
query_results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]})


print(f"Query:'{query}'\nNearest neighbors:")
print(query_results) # NOTE: Your results might look slightly different to ours.

Query:'Comment les chaînes de règles peuvent être utilisées pour déclencher des actions?'
Nearest neighbors:
                                                                                                                                                                                                                                             texts  \
15                                                                                           * Déclencher des actions : Les chaînes de règles peuvent être utilisées pour déclencher des actions  basées sur des données provenant d'appareils IoT   
9                                                                           Chaque nœud de règle d'une chaîne de règles exécute une tâche spécifique, comme le  filtrage de données, la transformation de données ou le déclenchement d'une action   
11                                                                                 Elles peuvent être utilisées pour :  * Valider les données : Les chaîn

In [ ]:
#@title Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=20)
umap_embeds = reducer.fit_transform(embeds)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': df['text']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text']
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)